In [3]:
# Import packages ....
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
from bullet import Bullet, Check, YesNo, Input, SlidePrompt
from bullet import colors
import datetime
import csv
from tensorflow.keras.callbacks import TensorBoard
import ipywidgets as widgets
print("---------All packages are imported--------------")


---------All packages are imported--------------


In [59]:
model_nom = widgets.Text(
    value='model_noun',
    placeholder='Type model_noun',
    description='model_noun:',
    disabled=False
)

path = widgets.Text(
    value='./Dataset',
    placeholder='Type the path',
    description='path:',
    disabled=False
)

batch_size = widgets.IntText(
        value=128,
        description='batch_size :',
        disabled=False
)

epochs = widgets.IntText(
        value=10,
        description='epochs :',
        disabled=False
)

dropout1 = widgets.FloatText(
        value=0.2,
        description='dropout1 :',
        disabled=False
)

dropout2 = widgets.FloatText(
        value=0.2,
        description='dropout2 :',
        disabled=False
)

optimizer = widgets.Text(
    value='adam',
    placeholder='Type the optimizer',
    description='optimizer:',
    disabled=False
)

learningrate = widgets.FloatText(
        value=0.001,
        description='learningrate :',
        disabled=False
)

momentum = widgets.FloatText(
        value=0.001,
        description='momentum :',
        disabled=False
)

rho = widgets.FloatText(
        value=0.0,
        description='rho :',
        disabled=False
)

beta1 = widgets.FloatText(
        value=0.9,
        description='beta1 :',
        disabled=False
)

beta2 = widgets.FloatText(
        value=0.9,
        description='beta2 :',
        disabled=False
)

nesterov = widgets.Text(
    value='False',
    placeholder='Type True or False',
    description='T or F:',
    disabled=False
)

amsgrad = widgets.Text(
    value='False',
    placeholder='Type True or False',
    description='T or F:',
    disabled=False
)

Kfold = widgets.IntText(
        value=10,
        description='Kfold :',
        disabled=False
)
display(model_nom)
display(path)
display(batch_size)
display(epochs)
display(dropout1)
display(dropout2)
display(optimizer)
display(learningrate)
display(momentum)
display(rho)
display(beta1)
display(beta2)
display(nesterov)
display(amsgrad)
display(Kfold)

Text(value='model_noun', description='model_noun:', placeholder='Type model_noun')

Text(value='./Dataset', description='path:', placeholder='Type the path')

IntText(value=128, description='batch_size :')

IntText(value=10, description='epochs :')

FloatText(value=0.2, description='dropout1 :')

...

FloatText(value=0.9, description='beta1 :')

FloatText(value=0.9, description='beta2 :')

Text(value='False', description='T or F:', placeholder='Type True or False')

Text(value='False', description='T or F:', placeholder='Type True or False')

IntText(value=10, description='Kfold :')

In [60]:
n_split=Kfold.value
epochs=epochs.value
model_nom = model_nom.value
pathData = path.value
optimizer = optimizer.value
dropout1 = dropout1.value
dropout2 = dropout2.value
batch_size = batch_size.value
learningrate = learningrate.value
momentum = bool(momentum.value)
nesterov = bool(nesterov.value)
beta1 = beta1.value
beta2 = beta2.value
amsgrad = amsgrad.value

In [63]:
#create a folder of logs including models in orther to visualize them with tensorboard tool
tensorboard = TensorBoard(log_dir='logs/{}'.format(model_nom))

In [64]:
# create path data
PATH = os.path.join(pathData)

# define training and validation paths
train_dir = os.path.join(PATH, 'train')
validation_dir = os.path.join(PATH, 'validation')

# create a list 'train' containing paths to every class in train folder
train = []
for i in range(len(os.listdir(train_dir))):
    # directory with our training pictures
    train += [os.path.join(train_dir, os.listdir(train_dir)[i])]
    train = sorted(train)

# create a list 'validation' containing paths to every class in validation folder
validation = []
for i in range(len(os.listdir(validation_dir))):
    # directory with our validation pictures
    validation += [os.path.join(validation_dir,
                                os.listdir(validation_dir)[i])]
    validation = sorted(validation)

# Understand the data
total_train = 0
for i in range(len(os.listdir(train_dir))):
    # total_train = total number of images in all classes in train folder
    total_train += len(os.listdir(train[i]))

total_val = 0
for i in range(len(os.listdir(validation_dir))):
    # total_val =total number of images in all classes in validation folder
    total_val += len(os.listdir(validation[i]))

In [65]:
# define attributes
IMG_HEIGHT = 150
IMG_WIDTH = 150
RGB_channels = 3  # three colors: Red,Green & Blue
filters_for_CONV1 = 16  # the number of output filters in the convolution 1
filters_for_CONV2 = 32  # the number of output filters in the convolution 2
filters_for_CONV3 = 64  # the number of output filters in the convolution 3
Kernel_SIZE = 3  # An integer or tuple/list of 2 integers,
# specifying the height and width of the 2D convolution window.
# Can be a single integer to specify the same value for all spatial dimensions.
units1 = 512  # Positive integer, dimensionality of the output space
units2 = len(os.listdir(train_dir))
units3 = dropout1
units4 = dropout2

In [66]:
# Data preparation
# Generator for our training data
train_image_generator = ImageDataGenerator(rescale=1. / 255,
                                           rotation_range=45,
                                           width_shift_range=.15,
                                           height_shift_range=.15,
                                           horizontal_flip=True,
                                           zoom_range=0.5)

# Generator for our validation data
validation_image_generator = ImageDataGenerator(
    rescale=1. / 255)

In [67]:
#  read the images from train folder
train_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size,
                                                           directory=train_dir,
                                                           shuffle=True,
                                                           target_size=(
                                                               IMG_HEIGHT, IMG_WIDTH),
                                                           class_mode='categorical')

#  read the images from validation folder
val_data_gen = validation_image_generator.flow_from_directory(batch_size=batch_size,
                                                              directory=validation_dir,
                                                              target_size=(
                                                                  IMG_HEIGHT, IMG_WIDTH),
                                                              class_mode='categorical')

Found 2480 images belonging to 10 classes.
Found 100 images belonging to 10 classes.


In [ ]:
# When using any layer as the first layer in a model,
# provide the keyword argument input_shape (tuple of integers,
# does not include the sample axis), e.g. input_shape=(128, 128, 3)
# for 128x128 RGB pictures in data_format="channels_last".

# define convolutions & fully connected layers
model = Sequential([
    Conv2D(filters_for_CONV1, Kernel_SIZE, padding='same',
           activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, RGB_channels)),
    MaxPooling2D(),
    Conv2D(filters_for_CONV2, Kernel_SIZE, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(filters_for_CONV3, Kernel_SIZE, padding='same', activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dropout(units3),
    Dense(units1, activation='relu'),
    Dropout(units4),
    Dense(units2, activation='softmax')
])

# switch beteween different optimizers
if optimizer == 'sgd':
    dynamic_optimizer = tf.keras.optimizers.SGD(
        learning_rate=learningrate, momentum=momentum, nesterov=nesterov)
elif optimizer == 'rmsprop':
    dynamic_optimizer = tf.keras.optimizers.RMSprop(learning_rate=learningrate)
elif optimizer == 'adagrad':
    dynamic_optimizer = tf.keras.optimizers.Adagrad(learning_rate=learningrate)
elif optimizer == 'adadelta':
    dynamic_optimizer = tf.keras.optimizers.Adadelta(
        learning_rate=learningrate)
elif optimizer == 'adamax':
    dynamic_optimizer = tf.keras.optimizers.Adamax(
        learning_rate=learningrate, beta_1=beta1, beta_2=beta2)
elif optimizer == 'nadam':
    dynamic_optimizer = tf.keras.optimizers.Nadam(
        learning_rate=learningrate, beta_1=beta1, beta_2=beta2)
else:  # Adam
    dynamic_optimizer = tf.keras.optimizers.Adam(
        learning_rate=learningrate, beta_1=beta1, beta_2=beta2, amsgrad=amsgrad)

# build a model architecture with CNN
model.compile(optimizer=dynamic_optimizer,
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

# Summarize Model
# Keras provides a way to summarize a model.
# The summary is textual and includes information about: The layers and their order in the model.
model.summary()

# train our model to get all the paramters to the correct value so that we can map our inputs to our outputs.
history = model.fit_generator(
    train_data_gen,
    steps_per_epoch=total_train // batch_size,
    epochs=epochs,
    validation_data=val_data_gen,
    validation_steps=total_val // batch_size,
    callbacks=[tensorboard]
)

# save our model
model.save("Models/" + model_nom + ".h5")

In [ ]:
# save our model
model.save("Models/" + model_nom + ".h5")

# create a csv file to write report 
# (Timte, Model, Epochs, Batch Size, Number Classes, Avg Accuracy, Loss, Optimizer, Dropout, Learning Rate)
# This conditions to deal with the problem of 'acc' and 'accuracy' in different environements
with open('report.csv', 'a') as reportFile:
    if 'acc' in history.history:                     
        my_accuracy = history.history['acc'][0]
    if 'accuracy' in history.history:
        my_accuracy = history.history['accuracy'][0]

    if ('accuracy' in history.history) or ('acc' in history.history):
        x = datetime.datetime.now()
        info = [x, model_nom, epochs, batch_size, units2,
                my_accuracy, history.history['loss'][0], optimizer, dropout1, dropout2, learningrate]
        writer = csv.writer(reportFile, delimiter=',', lineterminator='\n')
        writer.writerow(info)
    else:
        print("Couldn't save the report!!")
    
    print(my_accuracy)  # Will help us in the K-fold cv script (K-fold.sh)